In [2]:
import ase.io
from LDHPbuilder.perovskite_builder import OrganicMolecule, InorganicMonolayer, PerovskiteBuilder

This Notebooks shows how to use LDHP builder to make a set of candidate structures for a 2D hybrid perovskite, and then relax them with a MACE potential.

# 1) Making Candidate Structures

### 1.1) Starting Information

Suppose you want to predict the structure formed when a given organic cation is combined with chosen inorganic framework. Currently, our code assumes that the inorganic frameowrk will be a monolayer of corner sharing octahedra of the form $BX_4$. To get started, you therefore need:
- a cation geometry in an `ase.Atoms` object, with a known charge
- a choice of $B$-site
- a choice of $X$-site
- an indended unit cell size to search over

In [3]:
# load the geometry of an organic cation, for instance from an xyz file:
cation_atoms = ase.io.read('cation_plus_2.xyz')

### 1.2) Construct an `InorganicMonolayer` object and an `OrganicMolecule` object

An `InorganicMonolayer` is an atoms object of a monolayer in a wrapper which contains some useful information for structure generation. You can specify the size of the unit cell in the in-plane directions, through the parameter `num_unit_cell_octahedra`.

Similarly, an `OrganicMolecule` object wraps the geometry of the molecule with some useful information and functions. The charge must be specified manually.

In [8]:
# we will make  monolayer of PbI_4 octahedra, with 2 lead atoms per unit cell
monolayer = InorganicMonolayer.from_species_specification('Pb', 'I', num_unit_cell_octahedra=2)

molecule = OrganicMolecule(
    cation_atoms,
    2 # charge - this cation is doubly charged
)

### 1.3) Generate Samples

In [9]:
pb = PerovskiteBuilder()

samples = pb.generate_homogeneous_perovskite_samples( # homogeneous since just one kind of molecule
    monolayer, 
    molecule, 
    num_layers=1, # should be a power of 2, but 4 already implies a very large search task
    num_samples=100, 
    max_num_attempts=150, 
    stacking_method='total_thickness' # this can be either 'total_thickness' or 'half_thickness'. use half_thickness for long, thin +1 molecules
)

# save the samples to a file to inspect them
# ase.io.write('samples.xyz', samples)

found 1 structures after 0 attempts
found 2 structures after 1 attempts
found 3 structures after 2 attempts
found 4 structures after 3 attempts
found 5 structures after 4 attempts
found 6 structures after 5 attempts
found 7 structures after 6 attempts
found 8 structures after 7 attempts
found 9 structures after 8 attempts
found 10 structures after 9 attempts
found 11 structures after 10 attempts
found 12 structures after 11 attempts
found 13 structures after 12 attempts
found 14 structures after 13 attempts
found 15 structures after 14 attempts
found 16 structures after 15 attempts
found 17 structures after 16 attempts
found 18 structures after 17 attempts
found 19 structures after 18 attempts
found 20 structures after 19 attempts
found 21 structures after 20 attempts
found 22 structures after 21 attempts
found 23 structures after 22 attempts
found 24 structures after 23 attempts
found 25 structures after 24 attempts
found 26 structures after 25 attempts
found 27 structures after 26 at

# 2) Relaxing Structures with the MACE calculator

In [ ]:
# TODO